# Assignment 5: Optimization
*<span style="float:right;">Norine NDOUDI</span>*


In [46]:
# Generating a toy dataset.
# DO NOT MODIFY THIS PART

import numpy as np
import math
import random as rand

paras = list((rand.random() - 0.5 for _ in range(9)))

def y_gen(x):
    h_11 = math.tanh(paras[0] * x[0] + paras[1] * x[1] + paras[2])
    h_12 = math.tanh(paras[3] * x[0] + paras[4] * x[1] + paras[5])
    h_21 = 1/(1 + np.exp(-(paras[6] * h_11 + paras[7] * h_12 + paras[8])))
    return h_21 + ((rand.random()-0.5)/100 if rand.random()>0.6 else 0)

n = 300
x = list(zip((rand.random() - 0.5 for _ in range(n)), (rand.random() - 0.5 for _ in range(n))))
y = list(map(y_gen, x))
y = [(i-min(y))/(max(y)-min(y)) for i in y]

In [47]:
# Spliting dataset into training, validation, and test.
# DO NOT MODIFY THIS PART

from sklearn.model_selection import train_test_split

r = 0.25
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=r)

x_val = x_train[:int(n*r)]
partial_x_train = x_train[int(n*r):]
y_val = y_train[:int(n*r)]
partial_y_train = y_train[int(n*r):]

In [48]:
# DO NOT MODIFY THIS PART

from keras import models
from keras import layers
from keras import optimizers

def create_model(learning_rate=0.01, beta_1=0.9, beta_2=0.999):
    model = models.Sequential()
    model.add(layers.Dense(2, activation='tanh', input_shape=(2,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2), loss='mse')
    return model

I used this [website page](https://adriangb.com/scikeras/stable/generated/scikeras.wrappers.KerasRegressor.html) about KerasRegressor to know how this function works, the same about [GridSearchCV function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [this one](https://www.kaggle.com/code/code1110/houseprice-data-cleaning-visualization/notebook) showing GridSearchCV with Keras using KerasRegressor. 

We will use a default value of epoch for the majority of hyperparameter combinations instead of testing different values of epochs for each hyperparameter combination. So we will include the number of epoch in KerasRegressor function and not in the param grid of GridSearchCV.  
I chose a number of 30 epochs because of the runtime but also I found that the loss had already reached a satisfactory level, so I don't need an high value of epoch.

In [49]:
# Complete the cell to find a best hyper-parameter setting using GridSerachCV

#keras.wrapper wasn't working and is deprecated. The module wasn't found so I used the recent one with scikeras
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

#We put the hyperparameters in the create_model function's parameters but without putting them into KerasRegressor element the code gave me an error
model = KerasRegressor(model=create_model, learning_rate=0.01, beta_1=0.9, beta_2=0.999, epochs=30, verbose=0)
    

In [50]:
#Three values for each hyper-parameters
learning_rate = [0.01, 0.001, 0.0001]
beta_1 = [0.9, 0.8, 0.7]
beta_2 = [0.999, 0.99, 0.9]
param_grid = dict(learning_rate=learning_rate, beta_1=beta_1, beta_2 = beta_2)

gridSearch = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')
gridSearch.fit(partial_x_train, partial_y_train)


GridSearchCV(estimator=KerasRegressor(beta_1=0.9, beta_2=0.999, epochs=30, learning_rate=0.01, model=<function create_model at 0x000001EA5593D3A0>, verbose=0),
             param_grid={'beta_1': [0.9, 0.8, 0.7],
                         'beta_2': [0.999, 0.99, 0.9],
                         'learning_rate': [0.01, 0.001, 0.0001]},
             scoring='neg_mean_squared_error')

In [51]:
#Print out the ‘negative mean squared error’ and the corresponding hyper-parameters
results = gridSearch.cv_results_
for mean, params in zip(results['mean_test_score'], results['params']):
    print(f"{mean} {params}")

#Print out the best result we had and the correspondinf hyper-parameters
print("\nBest : " f"{gridSearch.best_score_} {gridSearch.best_params_}")

-0.013875636505261501 {'beta_1': 0.9, 'beta_2': 0.999, 'learning_rate': 0.01}
-0.025365195413928326 {'beta_1': 0.9, 'beta_2': 0.999, 'learning_rate': 0.001}
-0.053621081520078905 {'beta_1': 0.9, 'beta_2': 0.999, 'learning_rate': 0.0001}
-0.004442773888265942 {'beta_1': 0.9, 'beta_2': 0.99, 'learning_rate': 0.01}
-0.0443179138948208 {'beta_1': 0.9, 'beta_2': 0.99, 'learning_rate': 0.001}
-0.06015340160127379 {'beta_1': 0.9, 'beta_2': 0.99, 'learning_rate': 0.0001}
-0.0033266694153097293 {'beta_1': 0.9, 'beta_2': 0.9, 'learning_rate': 0.01}
-0.0436401639837973 {'beta_1': 0.9, 'beta_2': 0.9, 'learning_rate': 0.001}
-0.060781831405522714 {'beta_1': 0.9, 'beta_2': 0.9, 'learning_rate': 0.0001}
-0.011953090150496346 {'beta_1': 0.8, 'beta_2': 0.999, 'learning_rate': 0.01}
-0.05327233487954577 {'beta_1': 0.8, 'beta_2': 0.999, 'learning_rate': 0.001}
-0.039339530349926835 {'beta_1': 0.8, 'beta_2': 0.999, 'learning_rate': 0.0001}
-0.030246729942194385 {'beta_1': 0.8, 'beta_2': 0.99, 'learning_ra